In [29]:
# ============================================
# CELDA 1 - Imports y carga de archivos CSV
# ============================================

# !pip install pulp   # si no lo tienes, descomenta y corre una vez

import pandas as pd
import os
import pulp

BASE_PATH = "C:/Users/Adrian/Desktop/analitica_labo\data_expo_final/csv_modelo_constelacion_sunat_mincetur"

dim_calendario  = pd.read_csv(os.path.join(BASE_PATH, "dim_calendario.csv"))
dim_cuode       = pd.read_csv(os.path.join(BASE_PATH, "dim_cuode.csv"))
dim_pais        = pd.read_csv(os.path.join(BASE_PATH, "dim_pais.csv"))
dim_productos   = pd.read_csv(os.path.join(BASE_PATH, "dim_productos.csv"))
dim_via         = pd.read_csv(os.path.join(BASE_PATH, "dim_via.csv"))
fact_sunat      = pd.read_csv(os.path.join(BASE_PATH, "fact_comercio_sunat.csv"))
fact_mincetur   = pd.read_csv(os.path.join(BASE_PATH, "fact_exportaciones_MINCETUR.csv"))

for name, df in {
    "dim_calendario": dim_calendario,
    "dim_cuode": dim_cuode,
    "dim_pais": dim_pais,
    "dim_productos": dim_productos,
    "dim_via": dim_via,
    "fact_comercio_sunat": fact_sunat,
    "fact_exportaciones_MINCETUR": fact_mincetur
}.items():
    print(f"{name:40s} -> {df.shape}")

<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Adrian\AppData\Local\Temp\ipykernel_26648\249218012.py:11: SyntaxWarning: invalid escape sequence '\d'
  BASE_PATH = "C:/Users/Adrian/Desktop/analitica_labo\data_expo_final/csv_modelo_constelacion_sunat_mincetur"


dim_calendario                           -> (81, 6)
dim_cuode                                -> (5, 2)
dim_pais                                 -> (247, 3)
dim_productos                            -> (7669, 4)
dim_via                                  -> (9, 2)
fact_comercio_sunat                      -> (4180529, 9)
fact_exportaciones_MINCETUR              -> (1432499, 15)


In [30]:
# ============================================
# CELDA 2 - EDA sobre fact_exportaciones_MINCETUR
# ============================================

print("Rango de años MINCETUR:", fact_mincetur["anio"].min(), "->", fact_mincetur["anio"].max())

print("\nBloques comerciales (TOP 10):")
print(
    fact_mincetur["bloque_comercial"]
    .value_counts()
    .head(10)
)

print("\nRegiones MINCETUR (TOP 15):")
print(
    fact_mincetur["region_mincetur"]
    .value_counts()
    .head(15)
)

print("\nSectores (TOP 10):")
print(
    fact_mincetur["sector"]
    .value_counts()
    .head(10)
)

Rango de años MINCETUR: 2020 -> 2025

Bloques comerciales (TOP 10):
bloque_comercial
APEC                677962
Otro                269894
Comunidad Andina    213584
Unión Europea       199482
MERCOSUR             59596
EFTA                 11981
Name: count, dtype: int64

Regiones MINCETUR (TOP 15):
region_mincetur
Lima Metropolitana    797095
Callao                124919
Tacna                  65738
Piura                  59174
Ica                    53591
Arequipa               50005
La Libertad            38720
Lima Región            36432
Puno                   31149
Junín                  25118
Lambayeque             20760
Áncash                 20214
Cajamarca              16096
Cusco                  14633
Sin Asignar            13485
Name: count, dtype: int64

Sectores (TOP 10):
sector
Agropecuario          456979
Textil                303632
Metal mecánico        179274
Químico               173614
Varios                 73481
Minero                 58214
Pesquero            

In [ ]:
# ============================================
# CELDA 3 - Indicadores por región usando el 100% de los datos
# ============================================

# Trabajamos sobre una copia
df = fact_mincetur.copy()

# Aseguramos quedarnos solo con registros que tengan región
df = df[df["region_mincetur"].notna()].copy()

# Construimos indicadores regionales
region_stats = (
    df.groupby("region_mincetur", as_index=False)
      .agg(
          valor_total=("valor_millones", "sum"),
          toneladas_totales=("toneladas_metricas", "sum"),
          anios_distintos=("anio", "nunique"),
          n_registros=("anio", "count")
      )
)

# Evitar divisiones por cero
region_stats = region_stats[region_stats["toneladas_totales"] > 0].copy()

# Rentabilidad histórica por región
region_stats["valor_por_ton"] = region_stats["valor_total"] / region_stats["toneladas_totales"]

# Valor promedio anual (para ver tamaño económico de la región)
region_stats["valor_prom_anual"] = region_stats["valor_total"] / region_stats["anios_distintos"]

print("Número de regiones con datos:", region_stats.shape[0])
region_stats.head()

In [ ]:
# ============================================
# CELDA 4 - Score de potencial exportador por región
# ============================================

region_stats = region_stats.copy()

def min_max_norm(s):
    if s.max() == s.min():
        return s * 0  # todos iguales => 0
    return (s - s.min()) / (s.max() - s.min())

# Normalizamos valor promedio anual y valor por tonelada
region_stats["score_valor"] = min_max_norm(region_stats["valor_prom_anual"])
region_stats["score_rentabilidad"] = min_max_norm(region_stats["valor_por_ton"])

# Score combinado (se puede justificar en tu informe)
# 0.6 peso al tamaño económico, 0.4 a la rentabilidad
region_stats["score_total"] = (
    0.6 * region_stats["score_valor"] +
    0.4 * region_stats["score_rentabilidad"]
)

print("Top 10 regiones según score_total:")
region_stats.sort_values("score_total", ascending=False).head(10)
